<a href="https://colab.research.google.com/github/JohnVarkey/Deep-Learning/blob/master/Conv3d%20Model%2060%25%20acc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
!pip install kaggle

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d mohamedmustafa/real-life-violence-situations-dataset -p /content

100% 3.58G/3.58G [01:13<00:00, 23.2MB/s]



In [4]:
!unzip \*.zip

Archive:  real-life-violence-situations-dataset.zip
  inflating: Real Life Violence Dataset/NonViolence/NV_1.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_10.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_100.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_1000.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_101.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_102.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_103.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_104.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_105.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_106.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_107.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_108.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_109.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_11.mp4  
  inflating: Real Life Violence Dataset/N

In [5]:
!mkdir TrainData && cd TrainData && mkdir NonViolence && mkdir Violence && echo done
!mkdir TestData && cd TestData && mkdir NonViolence && mkdir Violence && echo done

done
done


In [6]:
!rm -r real-life-violence-situations-dataset.zip && echo done
 

done


In [7]:
import glob
from tqdm import tqdm
import cv2
import math


globpath=sorted(glob.glob("/content/Real Life Violence Dataset/*/*"))
skipcount=0
namecount=0
for vidfile in tqdm(globpath):
  filename=vidfile.split("/")[-2]
  if vidfile=="/content/Real Life Violence Dataset/NonViolence/NV_1.mp4" or vidfile=="/content/Real Life Violence Dataset/Violence/V_1.mp4":
     namecount=1
  vid=cv2.VideoCapture(vidfile)
  fps = vid.get(cv2.CAP_PROP_FPS)
  fps_rate=math.floor(fps/3)
  while True:
    ret,frame=vid.read()
    frameId = vid.get(1)
    if not ret:
      break
    if (frameId % math.floor(fps) != 0 or skipcount<fps_rate):
        skipcount+=1
        continue
    skipcount=0
    if namecount<1249:
      cv2.imwrite("/content/TrainData/{}/{}_{}.jpg".format(filename,filename[0],namecount),frame)
    else:
      cv2.imwrite("/content/TestData/{}/{}_{}.jpg".format(filename,filename[0],namecount),frame)
    
    namecount+=1
  vid.release()

print(namecount)

100%|██████████| 2000/2000 [08:47<00:00,  3.79it/s]

5119


In [29]:
from tweaked_ImageGenerator_v2 import ImageDataGenerator
datagen = ImageDataGenerator()
test_data=datagen.flow_from_directory('/content/TrainData/', target_size=(64, 64), batch_size=10, frames_per_step=5)
train_data=datagen.flow_from_directory('/content/TestData/', target_size=(64, 64), batch_size=10, frames_per_step=5)




Found 2496 images belonging to 2 classes.
Found 7619 images belonging to 2 classes.


In [28]:
!mkdir chkp
input_Shape=(5,64,64,3)


mkdir: cannot create directory ‘chkp’: File exists


In [ ]:

import keras
from keras.layers import Dense,Dropout
from keras.utils.vis_utils import plot_model 

layers = keras.layers
models = keras.models
losses = keras.losses
optimizers = keras.optimizers
metrics = keras.metrics
num_classes = 2
input_shapes = (100,100,3)
vg19 = keras.applications.vgg19.VGG19
base_model = vg19(include_top=False,input_shape=(100,100,3))
for layer in base_model.layers:
    layer.trainable = False
model = models.Sequential()
num_classes = 2
cnn = models.Sequential()
cnn.add(base_model)
cnn.add(layers.Flatten())
model = models.Sequential()
model.add(layers.TimeDistributed(cnn,  input_shape=(4, 100, 100, 3)))
model.add(layers.LSTM(40))
model.add(layers.Dense(13, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(num_classes, activation="sigmoid"))
adam = optimizers.Adam(0.001)

adam = optimizers.Adam(0.001)

model.compile(loss='binary_crossentropy', optimizer= adam, metrics=["accuracy"])
model.summary()

Using TensorFlow backend.


80142336/80134624 [==============================] - 7s 0us/step
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 4, 4608)           20024384  
_________________________________________________________________
lstm_1 (LSTM)                (None, 40)                743840    
_________________________________________________________________
dense_1 (Dense)              (None, 13)                533       
_________________________________________________________________
dropout_1 (Dropout)          (None, 13)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 28        
Total params: 20,768,785
Trainable params: 744,401
Non-trainable params: 20,024,384
_________________________________________________________________


In [ ]:
#accracy 57%
import keras
from keras.layers import Dense,Dropout
from keras.utils.vis_utils import plot_model 

layers = keras.layers
models = keras.models
losses = keras.losses
optimizers = keras.optimizers
metrics = keras.metrics


model=models.Sequential()
#model.add(layers.TimeDistributed(cnn,  input_shape=input_Shape))
model.add(layers.Conv3D(96,(1,8,8),strides=(1,3,3), input_shape=input_Shape))
model.add(layers.MaxPool3D(pool_size=(1,5,5), strides=(1,2,2)))
model.add(layers.Conv3D(384,(2,4,4)))
model.add(layers.Conv3D(200,(3,3,3),strides=(1,1,1)))
model.add(layers.MaxPool3D(pool_size=(1,2,2)))
model.add(layers.Conv3D(200,(1,3,3),strides=(1,1,1)))
model.add(layers.MaxPool3D(pool_size=(1,2,2)))
model.add(layers.TimeDistributed(layers.Flatten()))
model.add(layers.LSTM(1000, return_sequences=False))


#model.add(layers.LSTM(100))


#model.add(EarlyStopping(monitor='val_loss', patience=2))


model.add(Dense(1000, activation='relu'))
model.add(Dropout(.50))
model.add(Dense(250, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))

adam = optimizers.Adam(0.001)

model.compile(loss='binary_crossentropy', optimizer= adam, metrics=["accuracy"])
model.summary()

In [36]:

import keras
from keras.layers import LSTM,TimeDistributed, Input,Conv3D, MaxPooling3D, Flatten, Dense,Dropout, Activation
from keras.models import Sequential
from keras.optimizers import Adam


model = Sequential()

model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=input_Shape, padding='same'))


model.add(Activation('relu'))
model.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('softmax'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model.add(Dropout(0.25))
model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('softmax'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.load_weights('/content/drive/My Drive/training/chkp/conv3dlitev2.1_weights.33-0.70.hdf5')
adam = Adam(0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer=adam, metrics=['accuracy'])
model.summary()






Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_93 (Conv3D)           (None, 5, 64, 64, 32)     2624      
_________________________________________________________________
activation_45 (Activation)   (None, 5, 64, 64, 32)     0         
_________________________________________________________________
conv3d_94 (Conv3D)           (None, 5, 64, 64, 32)     27680     
_________________________________________________________________
activation_46 (Activation)   (None, 5, 64, 64, 32)     0         
_________________________________________________________________
max_pooling3d_69 (MaxPooling (None, 2, 22, 22, 32)     0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 2, 22, 22, 32)     0         
_________________________________________________________________
conv3d_95 (Conv3D)           (None, 2, 22, 22, 64)    

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import keras
from keras.layers import Dense,Dropout,Input,ConvLSTM2D, BatchNormalization, Conv3D, LeakyReLU
from keras.utils.vis_utils import plot_model 

layers = keras.layers
models = keras.models
losses = keras.losses
optimizers = keras.optimizers
metrics = keras.metrics


seq=models.Sequential()


#seq.add(layers.MaxPool3D(pool_size=(1,5,5)))

#seq.add(layers.MaxPool3D((1,3,3)))

seq.add(ConvLSTM2D(filters=50, kernel_size=(10,10),
                   padding='same', return_sequences=True,strides=(3, 3),input_shape=input_Shape))
seq.add(BatchNormalization())


seq.add(ConvLSTM2D(filters=50, kernel_size=(3, 3),
                   padding='same', return_sequences=True, dropout=0.25))
seq.add(BatchNormalization())

seq.add(Conv3D(filters=60, kernel_size=(3, 3, 3),
               activation='sigmoid',
                data_format='channels_last'))


seq.add(ConvLSTM2D(filters=60, kernel_size=(3, 3),
                   padding='same', return_sequences=True,
                   dropout=0.25))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=60, kernel_size=(3, 3),
                   padding='same', return_sequences=True,
                   dropout=0.25))
seq.add(BatchNormalization())

seq.add(Conv3D(filters=60, kernel_size=(1, 3, 3),
               activation='sigmoid',
               data_format='channels_last'))

seq.add(ConvLSTM2D(filters=60, kernel_size=(4, 4),
                   padding='same', return_sequences=True,
                   dropout=0.25))
seq.add(BatchNormalization())

seq.add(Conv3D(filters=30, kernel_size=(3, 3, 3),
               activation='sigmoid',
               data_format='channels_last'))

seq.add(layers.Flatten())
seq.add(Dense(900, activation='relu'))
seq.add(Dropout(.25))
seq.add(Dense(250, activation='relu'))
seq.add(Dropout(.25))
seq.add(Dense(64, activation='relu'))
seq.add(Dense(2, activation='softmax'))



adam =optimizers.Adam(learning_rate=0.001)
seq.compile(loss='binary_crossentropy', optimizer= adam, metrics=["accuracy"])
seq.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_1 (ConvLSTM2D)  (None, 5, 22, 22, 50)     1060200   
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 22, 22, 50)     200       
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, 5, 22, 22, 50)     180200    
_________________________________________________________________
batch_normalization_2 (Batch (None, 5, 22, 22, 50)     200       
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 3, 20, 20, 60)     81060     
_________________________________________________________________
conv_lst_m2d_3 (ConvLSTM2D)  (None, 3, 20, 20, 60)     259440    
_________________________________________________________________
batch_normalization_3 (Batch (None, 3, 20, 20, 60)    

In [20]:
import keras
from keras.layers import LSTM,TimeDistributed, Input,Conv3D, MaxPooling3D, Flatten, Dense,Dropout, Activation, LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam


input_x=Input(input_Shape)
initial_conv = Conv3D(16, kernel_size= (3, 3, 3), padding='same')(input_x)
initial_conv = LeakyReLU(alpha=.001)(initial_conv)

initial_conv = Conv3D(32, kernel_size= (3, 3, 3), padding='same')(initial_conv)
initial_conv = LeakyReLU(alpha=.001)(initial_conv)


###########################
# PARALLEL 1

conv1 = Conv3D(16, kernel_size=(1, 1, 1),padding='same')(initial_conv)
conv1 = LeakyReLU(alpha=.001)(conv1)
conv1 = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(conv1)
conv1 = Activation('softmax')(conv1)
conv1 = Conv3D(16, kernel_size=(1, 1, 1),padding='same')(conv1)
conv1 = LeakyReLU(alpha=.001)(conv1)
conv1 = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(conv1)

# conv1 = Conv3D(16, kernel_size=(3, 3, 3),padding='same')(conv1)
# conv1 = LeakyReLU(alpha=.001)(conv1)
conv1 = Activation('softmax')(conv1)
conv1 = Conv3D(16, kernel_size=(1, 1, 1),padding='same')(conv1)
conv1 = LeakyReLU(alpha=.001)(conv1)


conv1 = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(conv1)

# conv1 = Conv3D(8, kernel_size=(1, 1, 1),padding='same')(conv1)

#check it    
# conv1 = LeakyReLU(alpha=.001)(conv1)

# conv1 = Conv3D(1, kernel_size=(1, 1, 1),padding='same')(conv1)

##############################

##############################

#Parallel 2

conv2 = Conv3D(8, kernel_size=(1, 1, 1),padding='same')(initial_conv)
conv2 = LeakyReLU(alpha=.001)(conv2)
conv2 = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(conv2)
conv2 = Activation('softmax')(conv2)
conv2 = Conv3D(16, kernel_size=(1, 1, 1),padding='same')(conv2)
conv2 = LeakyReLU(alpha=.001)(conv2)
conv2 = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(conv2)

# conv2 = Conv3D(8, kernel_size=(3, 3, 3),padding='same')(conv2)
# conv2 = LeakyReLU(alpha=.001)(conv2)
conv2 = Activation('softmax')(conv2)
conv2 = Conv3D(16, kernel_size=(1, 1, 1),padding='same')(conv2)
conv2 = LeakyReLU(alpha=.001)(conv2)


conv2 = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(conv2)

# conv2 = Conv3D(4, kernel_size=(1, 1, 1),padding='same')(conv2)

# #check it    
# conv2 = LeakyReLU(alpha=.001)(conv2)

# conv2 = Conv3D(1, kernel_size=(1, 1, 1),padding='same')(conv2)

###################################


##############################

#Parallel 3

conv3 = Conv3D(10, kernel_size=(1, 1, 1),padding='same')(initial_conv)
conv3 = LeakyReLU(alpha=.001)(conv3)
conv3 = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(conv3)
conv3 = Conv3D(10, kernel_size=(1, 1, 1),padding='same')(conv3)
conv3 = LeakyReLU(alpha=.001)(conv3)
conv3 = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(conv3)


# conv3 = Conv3D(4, kernel_size=(3, 3, 3),padding='same')(conv3)
# conv3 = LeakyReLU(alpha=.001)(conv3)

conv3 = Conv3D(16, kernel_size=(1, 1, 1),padding='same')(conv3)
conv3 = LeakyReLU(alpha=.001)(conv3)


conv3 = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(conv3)

# conv3 = Conv3D(4, kernel_size=(1, 1, 1),padding='same')(conv3)

# #check it    
# conv3 = LeakyReLU(alpha=.001)(conv3)

# conv3 = Conv3D(1, kernel_size=(1, 1, 1),padding='same')(conv3)

###################################

#parallel 4

conv4=Activation('relu')(initial_conv)
conv4=Conv3D(32, kernel_size=(3, 3, 3), padding='same')(conv4)
conv4=Activation('softmax')(conv4)
conv4=MaxPooling3D(pool_size=(3, 3, 3), padding='same')(conv4)
conv4=Dropout(0.25)(conv4)
conv4=Conv3D(64, kernel_size=(3, 3, 3), padding='same')(conv4)
conv4=Activation('relu')(conv4)
conv4=Conv3D(16, kernel_size=(3, 3, 3), padding='same')(conv4)
conv4=Activation('softmax')(conv4)
conv4=MaxPooling3D(pool_size=(3, 3, 3), padding='same')(conv4)




############################

added = keras.layers.Add()([conv1, conv2, conv3, conv4])

added = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(added)

added = Flatten()(added)

dense_1 = Dense(256)(added)

dense_2 = Dense(2)(dense_1)

model = Model(input_x, dense_2)

model.compile(loss="categorical_crossentropy",
              optimizer=Adam(), metrics=['accuracy']) 

model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 5, 32, 32, 3) 0                                            
__________________________________________________________________________________________________
conv3d_51 (Conv3D)              (None, 5, 32, 32, 16 1312        input_7[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_47 (LeakyReLU)      (None, 5, 32, 32, 16 0           conv3d_51[0][0]                  
__________________________________________________________________________________________________
conv3d_52 (Conv3D)              (None, 5, 32, 32, 32 13856       leaky_re_lu_47[0][0]             
____________________________________________________________________________________________

In [37]:
callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        '/content/chkp/conv3dlitev2.1_weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1),
]
history=model.fit_generator(
    train_data,
    validation_data=test_data,
    verbose=1,
    steps_per_epoch=761,
    epochs=50,
    validation_steps=249,
    callbacks=callbacks
)

Epoch 1/50
761/761 [==============================] - 392s 516ms/step - loss: 0.6348 - accuracy: 0.5681 - val_loss: 0.6689 - val_accuracy: 0.5325

Epoch 00001: saving model to /content/chkp/conv3dlitev2.1_weights.01-0.67.hdf5
Epoch 2/50
761/761 [==============================] - 399s 525ms/step - loss: 0.6328 - accuracy: 0.5684 - val_loss: 0.6414 - val_accuracy: 0.5350

Epoch 00002: saving model to /content/chkp/conv3dlitev2.1_weights.02-0.64.hdf5
Epoch 3/50
761/761 [==============================] - 395s 519ms/step - loss: 0.6304 - accuracy: 0.5647 - val_loss: 0.6601 - val_accuracy: 0.5298

Epoch 00003: saving model to /content/chkp/conv3dlitev2.1_weights.03-0.66.hdf5
Epoch 4/50
761/761 [==============================] - 394s 518ms/step - loss: 0.6269 - accuracy: 0.5721 - val_loss: 0.7444 - val_accuracy: 0.5284

Epoch 00004: saving model to /content/chkp/conv3dlitev2.1_weights.04-0.74.hdf5
Epoch 5/50
761/761 [==============================] - 397s 522ms/step - loss: 0.6288 - accuracy:

KeyboardInterrupt: ignored